In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,6.686620,True,18,1,43.0
1,9.360828,True,32,1,41.0
2,8.437091,False,30,1,39.0
3,6.124442,False,24,1,40.0
4,7.125340,False,26,1,41.0


In [3]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks
count,9989.000000,10000.000000,10000.000000,9890.000000
mean,7.297602,27.298900,1.034400,38.699798
std,1.291685,6.165838,0.192926,2.539957
min,0.612885,12.000000,1.000000,17.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.124034,32.000000,1.000000,40.000000
max,12.257702,50.000000,3.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5150
False    4850
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)
data.head()

,is_male,mother_age,plurality,gestation_weeks
39,1,32,1,41.0
6132,0,28,1,30.0
5986,0,44,1,38.0
7682,0,34,1,38.0
4910,1,31,1,40.0


In [7]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [8]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

In [9]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,433
Trainable params: 2,433
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
209/209 [==============================] - 1s 6ms/step - loss: 1.5894 - mae: 0.9899 - mse: 1.5894 - val_loss: 1.3083 - val_mae: 0.8954 - val_mse: 1.3083
Epoch 2/10
209/209 [==============================] - 1s 4ms/step - loss: 1.4373 - mae: 0.9453 - mse: 1.4373 - val_loss: 1.1398 - val_mae: 0.8224 - val_mse: 1.1398
Epoch 3/10
209/209 [==============================] - 1s 6ms/step - loss: 1.3529 - mae: 0.9152 - mse: 1.3529 - val_loss: 1.3466 - val_mae: 0.9109 - val_mse: 1.3466
Epoch 4/10
209/209 [==============================] - 1s 6ms/step - loss: 1.3322 - mae: 0.9078 - mse: 1.3322 - val_loss: 1.4778 - val_mae: 0.9675 - val_mse: 1.4778
Epoch 5/10
209/209 [==============================] - 1s 5ms/step - loss: 1.2834 - mae: 0.8884 - mse: 1.2834 - val_loss: 1.1287 - val_mae: 0.8175 - val_mse: 1.1287
Epoch 6/10
209/209 [==============================] - 1s 4ms/step - loss: 1.2588 - mae: 0.8795 - mse: 1.2588 - val_loss: 1.0835 - val_mae: 0.8018 - val_mse: 1.0835
Epoch 7/10
209/2

In [12]:
# Generate predictions on test examples
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.8165126
Actual val:  6.93794738514

Predicted val:  6.976392
Actual val:  7.68751907594

Predicted val:  6.648657
Actual val:  5.56226287026

Predicted val:  6.7478557
Actual val:  8.2232423726

Predicted val:  6.7568636
Actual val:  9.18666245754

Predicted val:  7.3945827
Actual val:  7.3744626639

Predicted val:  7.259664
Actual val:  8.75676104664

Predicted val:  6.811509
Actual val:  7.4736706818

Predicted val:  6.947529
Actual val:  6.3118345610599995

Predicted val:  6.8026495
Actual val:  6.87621795178



In [14]:
# Use the What-If Tool
# Prepare data for the What-If Tool
wit_data = pd.concat([x_test, y_test], axis=1)
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [15]:
# Instantiate the What-If Tool
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…